In [1]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

In [2]:
dir = os.listdir('Images')
names = np.array(dir)
#print(names)

In [ ]:
image_size = (200, 200)
batch_size = 32

training = tf.keras.preprocessing.image_dataset_from_directory(
    "Images",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
validation = tf.keras.preprocessing.image_dataset_from_directory(
    "Images",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
#from IPython import display
#display.Image("Images/n02086079-Pekinese/n02086079_146.jpg")

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in training.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        
    ]
)
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


In [26]:

model = make_model(input_shape=image_size + (3,), num_classes=120)
keras.utils.plot_model(model, show_shapes=True)
model.summary()

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 200, 200, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 200, 200, 3)  0           sequential_1[1][0]               
__________________________________________________________________________________________________
conv2d_12 (Conv2D)   

In [13]:


callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    training, epochs=50, callbacks=callbacks, validation_data=validation,
)

Epoch 1/50
515/515 [==============================] - 102s 196ms/step - loss: 3.5490 - accuracy: 0.1363 - val_loss: 4.8401 - val_accuracy: 0.0522
Epoch 2/50
515/515 [==============================] - 97s 188ms/step - loss: 3.3213 - accuracy: 0.1768 - val_loss: 3.8024 - val_accuracy: 0.1193
Epoch 3/50
515/515 [==============================] - 96s 187ms/step - loss: 3.1296 - accuracy: 0.2077 - val_loss: 3.4142 - val_accuracy: 0.1747
Epoch 4/50
515/515 [==============================] - 97s 187ms/step - loss: 2.9159 - accuracy: 0.2484 - val_loss: 3.7050 - val_accuracy: 0.1608
Epoch 5/50
515/515 [==============================] - 96s 187ms/step - loss: 2.7281 - accuracy: 0.2889 - val_loss: 3.1867 - val_accuracy: 0.2021
Epoch 6/50
515/515 [==============================] - 96s 187ms/step - loss: 2.5630 - accuracy: 0.3199 - val_loss: 4.2344 - val_accuracy: 0.1528
Epoch 7/50
515/515 [==============================] - 97s 187ms/step - loss: 2.3965 - accuracy: 0.3513 - val_loss: 3.0488 - val_a

In [22]:
import numpy as np

from keras.preprocessing import image
 
# predicting images
path = "pug.jpeg"
img = image.load_img(path, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)

print(names[np.argmax(classes)])

n02110958-pug


In [25]:
best_model = keras.models.load_model("save_at_50.h5")
path = "pug.jpeg"
img = image.load_img(path, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = best_model.predict(images, batch_size=10)

print(names[np.argmax(classes)])

n02110958-pug
